In [1]:
import os
from rich import print
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from openai import OpenAI, completions

from settings import OpenAISettings

import dotenv


dotenv.load_dotenv(dotenv.find_dotenv())

console = Console()
def get_client() -> OpenAI:
    settings = OpenAISettings()
    client = OpenAI(
        api_key=settings.openai_api_key.get_secret_value(),
        base_url=str(settings.openai_base_url),
    )
    return client




def completions_variant():
    client = get_client()
    messages = []
    
    print("[bold cyan]Начало чата[/bold cyan]")
    

    while True:
        user_input = input(" Вы: ")

        if user_input.lower() == "/exit":
            print("\n[bold red]Чат завершён.[/bold red]")
            break

        

        if user_input.startswith("/system"):
            messages.append({"role": "system", "content": user_input})
            
        else:
            messages.append({"role": "user", "content": user_input})
        
        if user_input.lower() == "/clear":
           messages = [m for m in messages if m["role"] == "system"] 
           continue
        if user_input.lower() == "/clear system":
           messages = [m for m in messages if m["role"] == "user"] 
           continue
                
        completion = client.chat.completions.create(
        model="Qwen/Qwen3-Next-80B-A3B-Instruct",
        messages=messages,
            )

        assistant_reply = completion.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_reply})
        
        
        console.print(
        Panel.fit(
        str(messages),
        title="[bold cyan]История сообщений[/bold cyan]",
        border_style="cyan",
        width=console.width // 2,  
        padding=(1, 2),           
    )
)
        console.print() 


        md = Markdown(assistant_reply)
        console.print(
            Panel(
                md,
                title="[bold green]Ответ модели[/bold green]",
                border_style="green",
                
                padding=(1, 2),            
            )
        )
        console.print()
    
       

if __name__ == "__main__":
   
    completions_variant()
    
    

    # ЗАДАЧА:
    # сделать бесконечй цикл для диалога с моделью в одном чате с сохранением всего этого контекста
    # если пользователь вводит "/exit", то выйти из цикла
    # если вводит "/clear", то очистить историю, кроме system prompt
    # если вводит "/system какой-то текст", то всё что после /system - будет системный промптом
    # для задачи обязательно использовать rich console https://rich.readthedocs.io/en/latest/console.html
    # и panel для красоты https://rich.readthedocs.io/en/latest/panel.html
    # + форматирование через https://rich.readthedocs.io/en/latest/markdown.html


Начало чата

╭────────────────── История сообщений ──────────────────╮
│                                                       │
│  [{'role': 'user', 'content': 'дарова'}, {'role':     │
│  'assistant', 'content': 'Дарова! 😊  \nКак дела?     │
│  Чем могу помочь?'}]                                  │
│                                                       │
╰───────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Ответ модели ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Дарова! 😊                                                                                                     │
│  Как дела? Чем могу помочь?                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────── История сообщений ──────────────────╮
│                                                       │
│  [{'role': 'user', 'content': 'дарова'}, {'role':     │
│  'assistant', 'content': 'Дарова! 😊  \nКак дела?     │
│  Чем могу помочь?'}, {'role': 'system', 'content':    │
│  '/system отвечай на "дарова" - "дарова броуски"'},   │
│  {'role': 'assistant', 'content': 'дарова броуски     │
│  😎'}]                                                │
│                                                       │
╰───────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Ответ модели ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  дарова броуски 😎                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────── История сообщений ──────────────────╮
│                                                       │
│  [{'role': 'user', 'content': 'дарова'}, {'role':     │
│  'assistant', 'content': 'Дарова! 😊  \nКак дела?     │
│  Чем могу помочь?'}, {'role': 'system', 'content':    │
│  '/system отвечай на "дарова" - "дарова броуски"'},   │
│  {'role': 'assistant', 'content': 'дарова броуски     │
│  😎'}, {'role': 'user', 'content': 'как жизнь'},      │
│  {'role': 'assistant', 'content': 'дарова броуски 😎  │
│  \nЖизнь — как мем: то в плюсе, то в минусе, но всё   │
│  равно с юмором 😄  \nУ меня всё шикарно —            │
│  бот-помощник, как батарейка, всегда заряжен!  \nА у  │
│  тебя как? Дела идут, или уже в режиме "кофе +        │
│  паника"? ☕🤯'}]                                     │
│                                                       │
╰───────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Ответ модели ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  дарова броуски 😎                                                                                              │
│  Жизнь — как мем: то в плюсе, то в минусе, но всё равно с юмором 😄                                             │
│  У меня всё шикарно — бот-помощник, как батарейка, всегда заряжен!                                              │
│  А у тебя как? Дела идут, или уже в режиме "кофе + паника"? ☕🤯                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Чат завершён.